# Sequential single-point Bayesian Optimisation

In [1]:
import torch
from nubo.acquisition import ExpectedImprovement, UpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import lbfgsb
from nubo.test_functions import Hartmann6D
from nubo.utils import LatinHypercubeSampling, unnormalise
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood


# test function
func = Hartmann6D(minimise=False)
dims = func.dims
bounds = func.bounds

# training data
torch.manual_seed(1)
lhs = LatinHypercubeSampling(dims=dims)
x_train = lhs.maximin(points=dims*5)
x_train = unnormalise(x_train, bounds=bounds)
y_train = func(x_train)

# Bayesian optimisation loop
iters = 40

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = GaussianLikelihood()
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    mll = ExactMarginalLogLikelihood(likelihood=likelihood, model=gp)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, mll=mll, lr=0.1, steps=200)

    # specify acquisition function
    # acq = ExpectedImprovement(gp=gp, y_best=torch.max(y_train))
    acq = UpperConfidenceBound(gp=gp, beta=1.96**2)

    # optimise acquisition function
    x_new, _ = lbfgsb(func=acq, bounds=bounds, num_starts=5)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if y_new > torch.max(y_train[:-1]):
        print(f"New best at evaluation {len(y_train)}: \t Inputs: {x_new.numpy().reshape(dims).round(4)}, \t Outputs: {-y_new.numpy().round(4)}")

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {float(y_train[best_iter]):.4f}")


New best at evaluation 33: 	 Inputs: [0.3159 0.8291 0.     0.5973 1.     0.    ], 	 Outputs: [-2.3698]
New best at evaluation 39: 	 Inputs: [0.3975 0.8033 0.2752 0.5622 0.635  0.    ], 	 Outputs: [-2.8438]
New best at evaluation 42: 	 Inputs: [0.4225 0.8332 1.     0.5555 0.     0.    ], 	 Outputs: [-3.0419]
New best at evaluation 49: 	 Inputs: [0.4017 0.8468 1.     0.5813 0.     0.0085], 	 Outputs: [-3.1208]
New best at evaluation 51: 	 Inputs: [0.4026 0.8657 1.     0.5994 0.     0.0246], 	 Outputs: [-3.1584]
New best at evaluation 55: 	 Inputs: [0.4253 0.8895 1.     0.5895 0.     0.0399], 	 Outputs: [-3.1627]
New best at evaluation 57: 	 Inputs: [0.3971 0.9002 1.     0.5686 0.     0.0405], 	 Outputs: [-3.1823]
New best at evaluation 62: 	 Inputs: [0.4015 0.873  1.     0.5623 0.     0.0426], 	 Outputs: [-3.1861]
New best at evaluation 63: 	 Inputs: [0.4035 0.8879 1.     0.5812 0.     0.0375], 	 Outputs: [-3.1921]
New best at evaluation 64: 	 Inputs: [0.4045 0.8868 1.     0.5793 0.     